<a href="https://colab.research.google.com/github/smwongela/NLP_Transformers/blob/main/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.4 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 29.1 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 140 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 74.6 MB/s 
     |████████████████████████████████| 144 kB 70.1 MB/s 
     |████████████████████████████████| 271 kB 70.3 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import transformers
import datasets
from datasets import load_dataset
all_ds=datasets.list_datasets()
all_ds[:5]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus']

In [4]:
swa_dataset=load_dataset("swahili")
swa_dataset

Generating train split:   0%|          | 0/42069 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3371 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3372 [00:00<?, ? examples/s]

Dataset swahili downloaded and prepared to /root/.cache/huggingface/datasets/swahili/swahili/1.0.0/15bf1d99abb939f83b5da3c798ed55e9803b3ea430f06bf7e003bd073b60172a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 42069
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3371
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3372
    })
})

In [5]:
swa_dataset['train'][0]

{'text': 'taarifa hiyo ilisema kuwa ongezeko la joto la maji juu ya wastani katikati ya bahari ya UNK inaashiria kuwepo kwa mvua za el nino UNK hadi mwishoni mwa april ishirini moja sifuri imeelezwa kuwa ongezeko la joto magharibi mwa bahari ya hindi linatarajiwa kuhamia katikati ya bahari hiyo hali ambayo itasababisha pepo kutoka kaskazini mashariki kuvuma kuelekea bahari ya hindi'}

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [11]:
#divide into 10k samples and add progress bar using tqdm
from tqdm.auto import tqdm
text_data=[]
file_count=0

for sample in tqdm(swa_dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n',' ')
    text_data.append(sample)

    if len(text_data)==10_000:
        with open(f'./sample_data/swa_{file_count}.txt', 'w',encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count +=1

with open(f'./sample_data/swa_{file_count}.txt', 'w',encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))

  0%|          | 0/42069 [00:00<?, ?it/s]

**Tokenizer**

In [12]:
#Get a list to path to all our files
from pathlib import Path
#create a string from each path object we get
paths =[str(x) for x in Path('./sample_data').glob('*.txt')]
paths
paths[:5]

['sample_data/swa_4.txt',
 'sample_data/swa_1.txt',
 'sample_data/swa_2.txt',
 'sample_data/swa_3.txt',
 'sample_data/swa_0.txt']

 Training a **tokenizer**

In [14]:
#Tokenizer we are going to use is a Byte Level by Pair Encoder:-BPE
#For bert we use sentence piece encodings and we end up with Unknown token when we don't have a token for a specific word/new word
# With BPE we are breaking things into bytes, we don't actually need an unknown token anymore

from tokenizers import ByteLevelBPETokenizer
#initialize the tokenizer
tokenizer = ByteLevelBPETokenizer()
#train it
#We are using RoBERTa, vocab_size= for roberta is 52000, 30,000 means less training time, min -frequency indicates minimum number of times we want to see a word before we add it to our vocab ,
#We are also including RoBERTA special tokens: <s> start sequence token, <pad>  padding token, </s> end of line token, <unk> and the masking token <mask>
#might take time---try with first 500
tokenizer.train(files=paths, vocab_size=32_000, min_frequency=2, special_tokens =['<s>', '<pad>', '</s>','<unk>','<mask>'])


In [17]:
import os
#os.mkdir('kiswaERTa')
tokenizer.save_model('kiswaERTa')

['kiswaERTa/vocab.json', 'kiswaERTa/merges.txt']

In [19]:
from transformers import RobertaTokenizerFast

In [20]:
tokenizer = RobertaTokenizerFast.from_pretrained('kiswaERTa')

In [26]:
tokenizer('shemeji yangu kuja hapa saa one')

{'input_ids': [0, 822, 9489, 1570, 1260, 820, 1070, 6660, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}